In [2]:
%load_ext autoreload

%autoreload 2

from vae_dist.core.VAE import baselineVAEAutoencoder
from vae_dist.core.training import train
from vae_dist.dataset.dataset import FieldDataset
import torch 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'pytorch_lightning'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
root = "../../data/cpet/"
# load model to gpu
dataset_vanilla = FieldDataset(
    root, 
    transform=None, 
    augmentation=None, 
    device=device
    )


dataset_loader_full = torch.utils.data.DataLoader(dataset_vanilla, batch_size=40, shuffle=True)


In [ ]:
model = baselineVAEAutoencoder(
    irreps = None, # not used rn 
    in_channels = 3,
    out_channels = 16,
    kernel_size = 5,
    stride = 1,
    padding = 0,
    dilation = 1,
    groups = 1,
    bias = True,
    padding_mode = 'zeros',
    #padding_mode = 'constant',
    latent_dim = 4, # final vae hidden layer 
    num_layers = 2, # not used rn 
    hidden_dim = 32,
    activation = 'relu', # not used rn 
    dropout = 0.1, # not used rn 
    batch_norm = False, # not used rn 
    beta = 1.0,
    device = device, 
    loss = 'elbo'
)

In [ ]:
train(model, dataset_loader_full, device = device, epochs = 100)